In [344]:
def parse_input(is_test=True) -> list:
    file_name = "input-test.txt" if is_test else "input.txt"
    with open(file_name, "r") as file:
        return [list(line.strip()) for line in file]

# Part 1


In [345]:
def is_inbound(grid: list, r: int, c: int) -> bool:
    row_length = len(grid)
    col_length = len(grid[0])
    return 0 <= r < row_length and 0 <= c < col_length

In [346]:
def bfs(grid: list, is_visited: set, value: str, loc: tuple, region_row_col: set):
    current_row, current_col = loc[0], loc[1]

    if not is_inbound(grid, current_row, current_col):
        return

    if loc in is_visited:
        return

    if grid[current_row][current_col] != value:
        return

    region_row_col.add((current_row, current_col))
    is_visited.add((current_row, current_col))

    bfs(grid, is_visited, value, (current_row - 1, current_col), region_row_col)  # up
    bfs(grid, is_visited, value, (current_row + 1, current_col), region_row_col)  # down
    bfs(grid, is_visited, value, (current_row, current_col - 1), region_row_col)  # left
    bfs(grid, is_visited, value, (current_row, current_col + 1), region_row_col)  # right

In [347]:
def get_regions(grid: list) -> dict:
    # dict of region name: list of set of region row & cols
    regions = dict()
    is_visited = set()
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if (i, j) in is_visited:
                continue

            new_region_value = grid[i][j]
            new_region_row_col = set()
            new_region_row_col.add((i, j))

            bfs(grid, is_visited, new_region_value, (i - 1, j), new_region_row_col)  # up
            bfs(grid, is_visited, new_region_value, (i + 1, j), new_region_row_col)  # down
            bfs(grid, is_visited, new_region_value, (i, j - 1), new_region_row_col)  # left
            bfs(grid, is_visited, new_region_value, (i, j + 1), new_region_row_col)  # right

            if new_region_value not in regions:
                regions[new_region_value] = [new_region_row_col]
                continue

            regions[new_region_value].append(new_region_row_col)

    return regions

In [348]:
def get_area(s: set) -> int:
    return len(s)

In [349]:
def get_perimeter(s: set) -> int:
    result = 4 * len(s)

    for row_col in s:
        row, col = row_col[0], row_col[1]
        if (row - 1, col) in s:
            result -= 1
        if (row + 1, col) in s:
            result -= 1
        if (row, col - 1) in s:
            result -= 1
        if (row, col + 1) in s:
            result -= 1

    return result

In [350]:
def solve_part_1():
    grid = parse_input(is_test=False)
    regions = get_regions(grid)
    result = 0
    for _, v in regions.items():
        for s in v:
            result += get_area(s) * get_perimeter(s)

    return result

In [351]:
solve_part_1()

1533644

# Part 2


In [352]:
# if the plot of a region is at (0,0), all the possible corners are (-0.5, -0.5), (-0.5, 0.5), (0.5, 0.5), (0.5, -0.5)
def get_plot_corners(s: set) -> list:
    result = set()

    for row_col in s:
        corners = [(-0.5, -0.5), (-0.5, 0.5), (0.5, 0.5), (0.5, -0.5)]
        for corner in corners:
            result.add((row_col[0] + corner[0], row_col[1] + corner[1]))

    return result

In [353]:
# no. of sides == no. of corners
# so instead of counting sides, we count corners instead
def get_sides(s: set, plot_corners: set):
    result = 0

    for plot_corner_row, plot_corner_col in plot_corners:
        to_top_right_plot = (plot_corner_row - 0.5, plot_corner_col + 0.5)  # Q1
        to_top_left_plot = (plot_corner_row - 0.5, plot_corner_col - 0.5)  # Q2
        to_bottom_left_plot = (plot_corner_row + 0.5, plot_corner_col - 0.5)  # Q3
        to_bottom_right_plot = (plot_corner_row + 0.5, plot_corner_col + 0.5)  # Q4

        is_in_region = [to_top_right_plot in s, to_top_left_plot in s, to_bottom_left_plot in s, to_bottom_right_plot in s]

        if sum(is_in_region) == 3:
            result += 1

        if sum(is_in_region) == 2:
            if is_in_region == [True, False, True, False] or is_in_region == [False, True, False, True]:
                result += 2

        if sum(is_in_region) == 1:
            result += 1

    return result

In [354]:
def solve_part_2():
    grid = parse_input(is_test=False)
    regions = get_regions(grid)
    result = 0
    for _, v in regions.items():
        for s in v:
            plot_corners = get_plot_corners(s)
            result += get_area(s) * get_sides(s, plot_corners)

    return result

In [355]:
solve_part_2()

936718